In [89]:
import math
import sys
from typing import List
from functools import lru_cache, reduce, cmp_to_key
from itertools import starmap, accumulate, chain, islice, product, tee, pairwise
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
from importlib import reload
reload(arrays)
reload(trees)
reload(lists)
reload(graphs)
import arrays as ar
import trees as tr
import lists as ls
import graphs as g
from lists import Node
print()

In [82]:
def tree_distance(root, target):
    def search(r):
        if not r:
            return 0, 0

        fl, dl = search(r.left)
        fr, dr = search(r.right)
        return (
            fl
            and (fl + 1, max(fl + 1 + dr, dl))
            or fr
            and (fr + 1, max(fr + 1 + dl, dr))
            or (int(r.data == target), max(dl, dr) + 1)
        )

    return search(root)[1] - 1

In [62]:
tree_distance(t, 4)

3

In [90]:
t = tr.make_bfo('1 N 2 N 3 N 4 N 5 N 6')
t.display()

1     
 \    
 2    
  \   
  3   
   \  
   4  
    \ 
    5 
     \
     6
